In [9]:
import torch.optim
import torchvision
from torch import nn
from torch.utils.data import DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
import time

In [11]:
dataset_path = r'E:\file\python\MyCode\dataset\51.datasets'
train_data = torchvision.datasets.CelebA(root=dataset_path, split='train',target_type='attr',transform=torchvision.transforms.ToTensor(), download=False)
test_data = torchvision.datasets.CelebA(root=dataset_path, split='test',target_type='attr',transform=torchvision.transforms.ToTensor(), download=False)
train_data = Subset(train_data, range(1000))
test_data = Subset(test_data, range(1000))
train_loader = DataLoader(dataset=train_data, batch_size=100, shuffle=False)
test_loader = DataLoader(dataset=test_data,  batch_size=100, shuffle=False)

In [32]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.feature_process = Sequential(Conv2d(3, 30, kernel_size=50, stride=1, padding=0),
                                          MaxPool2d(2),
                                          Flatten(),
                                          Linear(30*84 * 64, 40)
                                          )



    def forward(self, x):
        x = self.feature_process(x)
        return x

cuda_available = torch.cuda.is_available()  # GPU是否可用
device = torch.device('cuda' if cuda_available else 'cpu')

demo = Model()
demo.to(device)  # 创建模型

print(train_data[0][1].size())

torch.Size([40])


In [35]:
loss_fn = nn.BCEWithLogitsLoss()  # 损失函数
loss_fn.to(device)
# 优化器
learn_rate = 1e-2
optim = torch.optim.SGD(demo.parameters(), lr=learn_rate)
# 训练次数
epochs = 2
total_train_step = 0

# 画板 tensorboard --logdir='.\\example\\data\\40.nn_Conv2d' --port=6012
write = SummaryWriter('..\\data\\51.base_celeba')

## 训练

In [36]:

for epoch in range(1, 1 + epochs):
    print(f'第{epoch}轮训练'.center(50, '-'))
    start_time = time.time()
    epochs_train_loss = 0  # 批次训练集总损失
    for data in train_loader:
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        # 卷积层 - 处理特征
        outputs = demo(imgs)
        # 损失函数层 - 计算损失
        loss = loss_fn(outputs, labels)
        # 优化器 - 算法优化参数
        optim.zero_grad()
        loss.backward()
        optim.step()
        # 累加损失
        epochs_train_loss += loss.item()
        total_train_step += 1
        if total_train_step % 100 == 0:
            print(total_train_step,epochs_train_loss)
            epochs_train_loss = 0

----------------------第1轮训练-----------------------


RuntimeError: result type Float can't be cast to the desired output type Long